### pobranie danych źródłowych z portalu IMGW dla wybranego roku, miesiąca i zakresu danych

In [151]:
import requests, zipfile, io

# file = requests.get('https://danepubliczne.imgw.pl/datastore/getfiledown/Arch/Telemetria/Meteo/2024/Meteo_2024-10.zip')
# zip = zipfile.ZipFile(io.BytesIO(file.content))
# zip.extractall("dane_imgw")

## wczytanie danych meteo do struktur danych modułu Pandas

In [152]:
import pandas as pd

files = {
    "B00202A": "dane_imgw/B00202A_2024_10.csv",
    "B00300S": "dane_imgw/B00300S_2024_10.csv",
    "B00305A": "dane_imgw/B00305A_2024_10.csv",
    "B00604S": "dane_imgw/B00604S_2024_10.csv",
    "B00606S": "dane_imgw/B00606S_2024_10.csv",
    "B00608S": "dane_imgw/B00608S_2024_10.csv",
    "B00702A": "dane_imgw/B00702A_2024_10.csv",
    "B00703A": "dane_imgw/B00703A_2024_10.csv",
    "B00714A": "dane_imgw/B00714A_2024_10.csv",
    "B00802A": "dane_imgw/B00802A_2024_10.csv",
}

dataframes = {}
for key, path in files.items():
    df = pd.read_csv(path, header=None, sep=";")
    df.columns = ["kodSH", "parametrSH", "czas", f"{key}"]
    df['czas'] = pd.to_datetime(df['czas'], format="%Y-%m-%d %H:%M")
    dataframes[key] = df

base_key = "B00202A"
base_df = dataframes[base_key][["kodSH", "czas"]] .drop_duplicates() 

for key, df in dataframes.items():
    if key == base_key:
        continue 
    trimmed = df[["kodSH", "czas", f"{key}"]]
    base_df = pd.merge(base_df, trimmed, on=["kodSH", "czas"], how="left")

base_df['data'] = base_df['czas'].dt.date
base_df['time'] = base_df['czas'].dt.time
base_df = base_df.drop(columns=["czas"])
base_df

,kodSH,B00300S,B00305A,B00604S,B00606S,B00608S,B00702A,B00703A,B00714A,B00802A,data,time
0,249190090,3.29,0.17,NaN,0.0,0.0,0.0,0.9,NaN,92.06,2024-10-01,00:00:00
1,249190090,3.16,0.11,NaN,NaN,0.0,0.0,0.2,NaN,91.97,2024-10-01,00:10:00
2,249190090,3.05,0.03,NaN,NaN,0.0,0.0,0.7,NaN,91.98,2024-10-01,00:20:00
3,249190090,3.01,-0.02,NaN,NaN,0.0,0.2,1.4,NaN,92.45,2024-10-01,00:30:00
4,249190090,3.07,0.00,NaN,NaN,0.0,0.1,1.5,NaN,92.50,2024-10-01,00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1166100,254150060,11.50,11.00,NaN,NaN,0.0,7.0,9.3,NaN,80.90,2024-10-30,18:10:00
1166101,254150060,11.40,10.60,NaN,NaN,0.0,6.7,9.2,NaN,80.90,2024-10-30,18:20:00
1166102,254150060,11.40,10.70,NaN,NaN,0.0,6.8,9.4,NaN,81.70,2024-10-30,18:30:00
1166103,254150060,11.40,10.90,NaN,NaN,0.0,6.2,8.7,NaN,82.00,2024-10-30,18:40:00


### wczytanie danych przestrzennych do struktur danych modułu GeoPandas

### koordynaty stacji

In [ ]:
import geopandas as gpd

effacility = gpd.read_file("dane_przestrzenne/effacility.geojson")
effacility = effacility[["ifcid", "name1", "geometry"]]
effacility.rename(columns={"name1": "name"}, inplace=True)
effacility.to_crs(epsg=4326, inplace=True)
effacility['x'] = effacility.apply(lambda x: x['geometry'].x, axis=1)
effacility['y'] = effacility.apply(lambda y: y['geometry'].y, axis=1)
effacility.drop(columns=['geometry'], inplace=True)
effacility

,ifcid,name,x,y
0,149180010,Krzyżanowice,18.287222,49.993611
1,149180020,Chałupki,18.327778,49.921944
2,149180030,Łaziska,18.443056,49.924444
3,149180040,Gołkowice,18.496389,49.925833
4,149180050,Zebrzydowice,18.613333,49.880000
...,...,...,...,...
2647,453220010,Jezioro Rajgrodzkie,22.668333,53.739722
2648,454170010,Jezioro Jasień,17.616111,54.305000
2649,454170020,Jezioro Raduńskie Górne,17.968889,54.230278
2650,454170030,Jezioro Łebsko,17.490000,54.740556


In [167]:
powiaty = powiaty = gpd.read_file("dane_przestrzenne/powiaty.shp")
powiaty = powiaty[["name", "geometry"]]
powiaty.head()

,name,geometry
0,złotoryjski,"POLYGON ((269083.616 370884.075, 269164.192 37..."
1,jeleniogórski,"POLYGON ((244044.052 332419.671, 244053.906 33..."
2,lubański,"POLYGON ((228421.669 366620.161, 228453.763 36..."
3,oleśnicki,"POLYGON ((377122.144 378970.426, 377361.467 37..."
4,bolesławiecki,"POLYGON ((235157.110 379452.665, 235210.213 37..."


In [168]:
wojewodztwa = gpd.read_file("dane_przestrzenne/woj.shp")
wojewodztwa[["name", "geometry"]]
wojewodztwa.head()

,id,gmlid,gmlidentif,gmlident_1,id_localid,id_namespa,id_version,beginlifes,endlifespa,country,name,national_c,national_l,national_1,national_2,upper_leve,id_upper_l,geometry
0,186,ID.f1ef3856-09ba-4e3d-af9d-a876794d570f,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,f1ef3856-09ba-4e3d-af9d-a876794d570f,PL.PZGIK.200/PRG.AU,20171009T220000Z,2017/10/09 22:00:00.000,None,PL,dolnośląskie,02,http://inspire.ec.europa.eu/codelist/Administr...,2ndOrder,województwo,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,2875,"POLYGON ((205901.102 343181.955, 205962.944 34..."
1,325,ID.c26354a4-3043-4ca9-9105-df6f0f9c1d93,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,c26354a4-3043-4ca9-9105-df6f0f9c1d93,PL.PZGIK.200/PRG.AU,20120925T220000Z,2012/09/25 22:00:00.000,None,PL,kujawsko-pomorskie,04,http://inspire.ec.europa.eu/codelist/Administr...,2ndOrder,województwo,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,2875,"POLYGON ((383436.881 613642.850, 383483.577 61..."
2,523,ID.a1e8cdc7-d26d-4982-946f-ff3e5082eecd,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,a1e8cdc7-d26d-4982-946f-ff3e5082eecd,PL.PZGIK.200/PRG.AU,20120925T220000Z,2012/09/25 22:00:00.000,None,PL,lubelskie,06,http://inspire.ec.europa.eu/codelist/Administr...,2ndOrder,województwo,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,2875,"POLYGON ((681015.792 420027.270, 681441.239 42..."
3,637,ID.42d2335f-bd81-491e-b93c-effe1bcab872,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,42d2335f-bd81-491e-b93c-effe1bcab872,PL.PZGIK.200/PRG.AU,20130708T220000Z,2013/07/08 22:00:00.000,None,PL,lubuskie,08,http://inspire.ec.europa.eu/codelist/Administr...,2ndOrder,województwo,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,2875,"POLYGON ((196217.270 448607.281, 196232.688 44..."
4,811,ID.7d7e4145-814d-4159-8d85-e91946e444c5,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,7d7e4145-814d-4159-8d85-e91946e444c5,PL.PZGIK.200/PRG.AU,20120925T220000Z,2012/09/25 22:00:00.000,None,PL,łódzkie,10,http://inspire.ec.europa.eu/codelist/Administr...,2ndOrder,województwo,http://iip.gugik.gov.pl/so/AU/AdministrativeUn...,2875,"POLYGON ((435569.180 387437.039, 435633.348 38..."


### użycie astrala

In [156]:
from astral import LocationInfo
from astral.sun import sun
import datetime

city = LocationInfo("Warsaw", "Poland", "Europe/Warsaw", 52.232222, 21.008333)
s = sun(city.observer, date=datetime.date(2021, 2, 14), tzinfo=city.timezone)
print(s["dawn"], s["sunrise"], s["noon"], s["sunset"], s["dusk"])

2021-02-14 06:17:41.086058+01:00 2021-02-14 06:53:35.458511+01:00 2021-02-14 11:50:08+01:00 2021-02-14 16:47:27.725619+01:00 2021-02-14 17:23:24.874494+01:00
